<a href="https://colab.research.google.com/github/JuanAntCB/Caffeine-nb-mod-for-AFL/blob/main/highresDocking_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using material from https://github.com/RosettaCommons/PyRosetta.notebooks/blob/master/student-notebooks/08.01-Ligand-Docking-XMLObjects.ipynb


In [5]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
import sys
import matplotlib
init()

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2023.20+release.dbbb5449bf9583f5b8d8f4151636d460bbeedba3 2023-05-16T11:06:33] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r349 2023.20+release.dbbb5449bf9 dbbb5449bf9583f5b8d8f4151636d460bbeedba3 http://www.pyrosetta.org 2023-05-16T11:06:33
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-980634811 seed_offset=0 real_seed=-980634811
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-980634811 RG_type=mt19937
PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2023.20+release.dbbb5449bf9583f5b8d8f4151636d460b

In [6]:
from google.colab import drive
import os

drive.flush_and_unmount()

drive.mount('/content/drive')

path = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb'
path_aft = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aft.sdf'
path_params = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/AFT.params'
drive_path = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs'


Mounted at /content/drive


In [9]:
ligand_params = path_params
flags = f"""
-ignore_unrecognized_res 1
-extra_res_fa {ligand_params}
"""
pyrosetta.distributed.init(flags)
pose = pyrosetta.io.pose_from_file(path)
scorefxn = pyrosetta.create_score_function("ref2015")

core.import_pose.import_pose: File '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) 'AFT' from the PDB components dictionary for residue type 'pdb_AFT'
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 12 atoms at position 121 in file /content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb. Best match rsd_type:  pdb_AFT
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue VAL:CtermProteinFull 238
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: sc

In [22]:
# Obtener el número de cadenas en la pose
num_chains = pose.num_chains()
print(f"Number of chains in the pose: {num_chains}")

# Imprimir los identificadores de las cadenas
for i in range(1, num_chains + 1):
    chain_id = pose.pdb_info().chain(i)
    print(f"Chain {i} ID: {chain_id}")

# O bien, dividir la pose por cadenas y verificar cada una
split_poses = pose.split_by_chain()
for chain_num, chain_pose in enumerate(split_poses, start=1):
    chain_id = chain_pose.pdb_info().chain(1)
    print(f"Chain {chain_num} ID: {chain_id}")

Number of chains in the pose: 3
Chain 1 ID: A
Chain 2 ID: A
Chain 3 ID: A
core.kinematics.FoldTree: delete_jump_seqpos: deleting jump 1
core.kinematics.FoldTree: delete_jump_seqpos: renumbering jump 2 to 1
core.kinematics.FoldTree: delete_jump_seqpos: deleting jump 1
core.kinematics.FoldTree: delete_jump_seqpos: renumbering jump 2 to 1
core.kinematics.FoldTree: delete_jump_seqpos: deleting jump 1
core.kinematics.FoldTree: delete_jump_seqpos: renumbering jump 2 to 1
Chain 1 ID: A
Chain 2 ID: A
Chain 3 ID: B


Ligand docking using ```XmlObjects``:

In [19]:
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>

    <SCOREFXNS>
        <ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep"/>
        <ScoreFunction name="hard_rep" weights="ligand"/>
        <ScoreFunction name="fa_standard" weights="ref2015.wts"/>
    </SCOREFXNS>

    <LIGAND_AREAS>
        <LigandArea name="inhibitor_dock_sc_A" chain="A" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
        <LigandArea name="inhibitor_final_sc_A" chain="A" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
        <LigandArea name="inhibitor_final_bb_A" chain="A" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>

        <LigandArea name="inhibitor_dock_sc_B" chain="B" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
        <LigandArea name="inhibitor_final_sc_B" chain="B" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
        <LigandArea name="inhibitor_final_bb_B" chain="B" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>
    </LIGAND_AREAS>

    <INTERFACE_BUILDERS>
        <InterfaceBuilder name="side_chain_for_docking_A" ligand_areas="inhibitor_dock_sc_A"/>
        <InterfaceBuilder name="side_chain_for_final_A" ligand_areas="inhibitor_final_sc_A"/>
        <InterfaceBuilder name="backbone_A" ligand_areas="inhibitor_final_bb_A" extension_window="3"/>

        <InterfaceBuilder name="side_chain_for_docking_B" ligand_areas="inhibitor_dock_sc_B"/>
        <InterfaceBuilder name="side_chain_for_final_B" ligand_areas="inhibitor_final_sc_B"/>
        <InterfaceBuilder name="backbone_B" ligand_areas="inhibitor_final_bb_B" extension_window="3"/>
    </INTERFACE_BUILDERS>

    <MOVEMAP_BUILDERS>
        <MoveMapBuilder name="docking_A" sc_interface="side_chain_for_docking_A" minimize_water="false"/>
        <MoveMapBuilder name="final_A" sc_interface="side_chain_for_final_A" bb_interface="backbone_A" minimize_water="false"/>

        <MoveMapBuilder name="docking_B" sc_interface="side_chain_for_docking_B" minimize_water="false"/>
        <MoveMapBuilder name="final_B" sc_interface="side_chain_for_final_B" bb_interface="backbone_B" minimize_water="false"/>
    </MOVEMAP_BUILDERS>

    <SCORINGGRIDS width="15">
        <ClassicGrid grid_name="classic_A" weight="1.0"/>
        <ClassicGrid grid_name="classic_B" weight="1.0"/>
    </SCORINGGRIDS>

    <MOVERS>
        <Transform name="transform_A" chain="A" box_size="7.0" move_distance="0.2" angle="20" cycles="30" repeats="1" temperature="5"/>
        <Transform name="transform_B" chain="B" box_size="7.0" move_distance="0.2" angle="20" cycles="30" repeats="1" temperature="5"/>
        <HighResDocker name="high_res_docker_A" cycles="6" repack_every_Nth="3" scorefxn="ligand_soft_rep" movemap_builder="docking_A"/>
        <HighResDocker name="high_res_docker_B" cycles="6" repack_every_Nth="3" scorefxn="ligand_soft_rep" movemap_builder="docking_B"/>
        <FinalMinimizer name="final_A" scorefxn="hard_rep" movemap_builder="final_A"/>
        <FinalMinimizer name="final_B" scorefxn="hard_rep" movemap_builder="final_B"/>
        <InterfaceScoreCalculator name="add_scores_A" chains="A" scorefxn="hard_rep" native='/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb'/>
        <InterfaceScoreCalculator name="add_scores_B" chains="B" scorefxn="hard_rep" native='/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb'/>
    </MOVERS>

    <PROTOCOLS>
        <Add mover_name="transform_A"/>
        <Add mover_name="transform_B"/>
        <Add mover_name="high_res_docker_A"/>
        <Add mover_name="high_res_docker_B"/>
        <Add mover_name="final_A"/>
        <Add mover_name="final_B"/>
        <Add mover_name="add_scores_A"/>
        <Add mover_name="add_scores_B"/>
    </PROTOCOLS>

</ROSETTASCRIPTS>
""").get_mover("ParsedProtocol")

protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep"/>
		<ScoreFunction name="hard_rep" weights="ligand"/>
		<ScoreFunction name="fa_standard" weights="ref2015.wts"/>
	</SCOREFXNS>
	<LIGAND_AREAS>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="A" cutoff="6.0" name="inhibitor_dock_sc_A"/>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="A" cutoff="6.0" name="inhibitor_final_sc_A"/>
		<LigandArea Calpha_restraints="0.3" add_nbr_radius="false" all_atom_mode="true" chain="A" cutoff="7.0" name="inhibitor_final_bb_A"/>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="B" cutoff="6.0" name="inhibitor_dock_sc_B"/>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="B" cutoff="6.0" name="inhibitor_fin

Produce 5 global ligand docking trajectories using PyJobDistributor:

In [20]:
if not os.getenv("DEBUG"):
    working_dir = os.getcwd()
    output_dir = "outputs"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    os.chdir(output_dir)

    jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="test_lig_XMLObjects",
                                                              nstruct=5,
                                                              scorefxn=scorefxn)
    jd.native_pose = pose
    df = pd.DataFrame()
    while not jd.job_complete:
        test_pose = pose.clone()
        xml.apply(test_pose)
        test_df = pd.DataFrame.from_records(dict(test_pose.scores), index=[jd.current_name])
        df = df.append(test_df)
        jd.output_decoy(test_pose)
    os.chdir(working_dir)

Working on decoy: test_lig_XMLObjects_0.pdb
protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN MOVER Transform - transform_A=======================
protocols.rosetta_scripts.ParsedProtocol: [ ERROR ] Exception while processing protocol: 

File: /home/benchmark/rosetta/source/src/core/pose/chains_util.cc:455
chain_id A represents more than one chain!


RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/pose/chains_util.cc:455
chain_id A represents more than one chain!